In [1]:
from collections import Counter

In [14]:
import pandas as pd
from scipy.sparse import dok_matrix, save_npz
import numpy as np
import pickle
from tqdm import tqdm
import os

def build_profession_skill_matrix(csv_file_path, output_dir, chunk_size=50000):
    """Построение матрицы профессии-навыки из CSV файла"""
    os.makedirs(output_dir, exist_ok=True)

    # 1. Первый проход: сбор уникальных значений
    unique_professions = set()
    unique_skills = set()

    print("🔍 Первый проход: сбор уникальных профессий и навыков...")
    reader = pd.read_csv(csv_file_path,
                        dtype={'_id': str},
                        chunksize=chunk_size,
                        encoding='utf-8-sig')

    for chunk in tqdm(reader):
        # Сбор уникальных профессий с нормализацией
        professions = chunk['best_profession'].dropna().str.strip().unique()
        unique_professions.update(professions)

        # Обработка hard skills с нормализацией
        if 'hard_skills' in chunk:
            hard_skills = chunk['hard_skills'].dropna()
            hard_skills = hard_skills.str.split(';').explode()
            hard_skills = hard_skills.str.strip()
            hard_skills = hard_skills[hard_skills != '']
            unique_skills.update(hard_skills)

        # Обработка soft skills с нормализацией и префиксом
        if 'soft_skills' in chunk:
            soft_skills = chunk['soft_skills'].dropna()
            soft_skills = soft_skills.str.split(';').explode()
            soft_skills = soft_skills.str.strip()
            soft_skills = soft_skills[soft_skills != '']
            unique_skills.update("SOFT_" + soft_skills)

    # 2. Создание словарей индексов
    profession_to_idx = {prof: idx for idx, prof in enumerate(unique_professions)}
    skill_to_idx = {skill: idx for idx, skill in enumerate(unique_skills)}

    # 3. Второй проход: построение матрицы
    matrix = dok_matrix((len(unique_professions), len(unique_skills)), dtype=np.int32)

    print("\n🔧 Второй проход: построение матрицы...")
    reader = pd.read_csv(csv_file_path,
                        dtype={'_id': str},
                        chunksize=chunk_size,
                        encoding='utf-8-sig')

    skipped_skills = set()
    for chunk in tqdm(reader):
        for _, row in chunk.iterrows():
            if pd.isna(row['best_profession']):
                continue

            profession = str(row['best_profession']).strip()
            if not profession or profession not in profession_to_idx:
                continue

            p_idx = profession_to_idx[profession]

            # Обработка hard skills
            if pd.notna(row['hard_skills']):
                for skill in str(row['hard_skills']).split(';'):
                    if skill := skill.strip():
                        if skill in skill_to_idx:
                            s_idx = skill_to_idx[skill]
                            matrix[p_idx, s_idx] += 1
                        else:
                            skipped_skills.add(skill)

            # Обработка soft skills
            if pd.notna(row['soft_skills']):
                for skill in str(row['soft_skills']).split(';'):
                    if skill := skill.strip():
                        skill_key = "SOFT_" + skill
                        if skill_key in skill_to_idx:
                            s_idx = skill_to_idx[skill_key]
                            matrix[p_idx, s_idx] += 1
                        else:
                            skipped_skills.add(skill_key)

    # Сообщение о пропущенных навыках
    if skipped_skills:
        print(f"\n⚠️ Пропущено {len(skipped_skills)} навыков, отсутствующих в словаре")
        print("Примеры пропущенных навыков:", list(skipped_skills)[:5])

    # 4. Сохранение результатов
    print("\n💾 Сохранение результатов...")
    csr_matrix = matrix.tocsr()
    save_npz(os.path.join(output_dir, "profession_skills_matrix.npz"), csr_matrix)

    # Сохранение словарей
    with open(os.path.join(output_dir, "profession_to_idx.pkl"), 'wb') as f:
        pickle.dump(profession_to_idx, f)

    with open(os.path.join(output_dir, "skill_to_idx.pkl"), 'wb') as f:
        pickle.dump(skill_to_idx, f)

    # Обратные словари
    idx_to_profession = {v: k for k, v in profession_to_idx.items()}
    idx_to_skill = {v: k for k, v in skill_to_idx.items()}

    with open(os.path.join(output_dir, "idx_to_profession.pkl"), 'wb') as f:
        pickle.dump(idx_to_profession, f)

    with open(os.path.join(output_dir, "idx_to_skill.pkl"), 'wb') as f:
        pickle.dump(idx_to_skill, f)

    print(f"✅ Готово! Матрица размером {csr_matrix.shape[0]} профессий × {csr_matrix.shape[1]} навыков")
    print(f"📁 Результаты сохранены в: {output_dir}")

# Пример использования

build_profession_skill_matrix(
    csv_file_path="../6_framework/results/result.csv",
    output_dir="output_matrix",
    chunk_size=100000
)

🔍 Первый проход: сбор уникальных профессий и навыков...


1it [00:00, 151.71it/s]



🔧 Второй проход: построение матрицы...


1it [00:00, 49.47it/s]


💾 Сохранение результатов...
✅ Готово! Матрица размером 24 профессий × 345 навыков
📁 Результаты сохранены в: output_matrix


In [8]:
!pip install graphistry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.4/332.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 26.4 MB/s eta 0:00:00


In [10]:
import numpy as np
from scipy.sparse import load_npz
import pandas as pd
import graphistry
import pickle
from tqdm import tqdm

# Загрузка данных
matrix = load_npz("output_matrix/profession_skills_matrix.npz")
with open("output_matrix/skill_to_idx.pkl", "rb") as f:
    skill_to_idx = pickle.load(f)
with open("output_matrix/idx_to_skill.pkl", "rb") as f:
    idx_to_skill = pickle.load(f)

# Конвертация в CSC для эффективного доступа по столбцам (навыкам)
csc_matrix = matrix.tocsc()

# Расчет общей статистики
total_professions = matrix.shape[0]
skill_frequencies = np.array(csc_matrix.sum(axis=0)).flatten()

# Функция расчета NPMI
def calculate_npmi(skill_i, skill_j):
    # Совместная встречаемость
    col_i = csc_matrix[:, skill_i]
    col_j = csc_matrix[:, skill_j]
    co_occurrence = col_i.multiply(col_j).sum()

    if co_occurrence == 0:
        return 0.0

    # Вероятности
    p_i = skill_frequencies[skill_i] / total_professions
    p_j = skill_frequencies[skill_j] / total_professions
    p_ij = co_occurrence / total_professions

    # PMI и NPMI
    pmi = np.log(p_ij / (p_i * p_j))
    npmi = pmi / (-np.log(p_ij))

    return npmi

In [15]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
from tqdm import tqdm
import os
import math
from scipy.sparse import load_npz

# Загрузка предварительно вычисленных данных
matrix = load_npz("output_matrix/profession_skills_matrix.npz")
with open("output_matrix/skill_to_idx.pkl", "rb") as f:
    skill_to_idx = pickle.load(f)
with open("output_matrix/idx_to_skill.pkl", "rb") as f:
    idx_to_skill = pickle.load(f)
with open("output_matrix/profession_to_idx.pkl", "rb") as f:
    profession_to_idx = pickle.load(f)

# Вычисление частот
total_professions = matrix.shape[0]
skill_frequencies = np.array(matrix.sum(axis=0)).flatten()
profession_frequencies = np.array(matrix.sum(axis=1)).flatten()
# Функция расчета NPMI
def calculate_npmi(freq_i, freq_j, co_occurrence, total):
    if co_occurrence == 0:
        return 0.0

    p_i = freq_i / total
    p_j = freq_j / total
    p_ij = co_occurrence / total

    # Избегаем нулевых вероятностей
    if p_i <= 0 or p_j <= 0 or p_ij <= 0:
        return 0.0

    pmi = math.log(p_ij / (p_i * p_j))
    npmi = pmi / (-math.log(p_ij))

    return npmi

# Инициализация структур данных
file_path = "../6_framework/results/result.csv"
chunk_size = 100000
min_cooccurrence = 2  # Минимальная совместная встречаемость

print("⚙️ Инициализация завершена")

⚙️ Инициализация завершена


In [20]:
# Сбор статистики для NPMI
from itertools import combinations
soft_soft_edges = Counter()
soft_hard_edges = Counter()
profession_soft_edges = Counter()

soft_freq = Counter()
hard_freq = Counter()
profession_freq = Counter()
total_vacancies = 0

# Первый проход: сбор частот
print("🔍 Первый проход: сбор статистики...")
reader = pd.read_csv(file_path,
                        dtype={'_id': str},
                        chunksize=chunk_size,
                        encoding='utf-8-sig')


for chunk in tqdm(reader):
    for _, row in chunk.iterrows():
        total_vacancies += 1
        profession = str(row['best_profession']).strip() if pd.notna(row['best_profession']) else None
        if profession:
            profession_freq[profession] += 1

        # Hard skills
        hard_skills = []
        if pd.notna(row['hard_skills']):
            for skill in str(row['hard_skills']).split(';'):
                if skill := skill.strip():
                    hard_skills.append(skill)
                    hard_freq[skill] += 1

        # Soft skills
        soft_skills = []
        if pd.notna(row['soft_skills']):
            for skill in str(row['soft_skills']).split(';'):
                if skill := skill.strip():
                    soft_skill = "SOFT_" + skill
                    soft_skills.append(soft_skill)
                    soft_freq[soft_skill] += 1

        # Soft-soft связи
        '''for i in range(len(soft_skills)):
            for j in range(i + 1, len(soft_skills)):
                key = tuple(sorted([soft_skills[i], soft_skills[j]]))
                soft_soft_edges[key] += 1'''
        for skill_pair in combinations(soft_skills, 2):
          key = tuple(sorted(skill_pair))
          soft_soft_edges[key] += 1
        # Soft-hard связи
        for soft in soft_skills:
            for hard in hard_skills:
                key = (soft, hard)
                soft_hard_edges[key] += 1

        # Profession-soft связи
        if profession:
            for soft in soft_skills:
                key = (profession, soft)
                profession_soft_edges[key] += 1

# Фильтрация редких связей
soft_soft_edges = {k: v for k, v in soft_soft_edges.items() if v >= min_cooccurrence}
soft_hard_edges = {k: v for k, v in soft_hard_edges.items() if v >= min_cooccurrence}
profession_soft_edges = {k: v for k, v in profession_soft_edges.items() if v >= min_cooccurrence}

# Расчет NPMI и создание DataFrame
def create_npmi_df(edges_dict, freq_x, freq_y):
    rows = []
    for (x, y), co_occur in edges_dict.items():
        npmi = calculate_npmi(freq_x[x], freq_y[y], co_occur, total_vacancies)
        rows.append({
            "source": x,
            "target": y,
            "co_occurrence": co_occur,
            "npmi": npmi
        })
    return pd.DataFrame(rows)

# Создание DataFrame с NPMI
print("\n📊 Расчет NPMI для графов...")
df_soft_soft = create_npmi_df(
    soft_soft_edges,
    soft_freq,
    soft_freq
)

df_soft_hard = create_npmi_df(
    soft_hard_edges,
    soft_freq,
    hard_freq
)

df_profession_soft = create_npmi_df(
    profession_soft_edges,
    profession_freq,
    soft_freq
)

print("✅ Графы построены:")
print(f"Soft-Soft: {len(df_soft_soft)} ребер")
print(f"Soft-Hard: {len(df_soft_hard)} ребер")
print(f"Profession-Soft: {len(df_profession_soft)} ребер")

🔍 Первый проход: сбор статистики...


1it [00:00, 51.39it/s]


📊 Расчет NPMI для графов...
✅ Графы построены:
Soft-Soft: 35 ребер
Soft-Hard: 186 ребер
Profession-Soft: 19 ребер


In [21]:
def write_to_file(soft_soft, soft_hard,profession_soft, nodes):
    soft_soft.to_csv('df_soft_soft', encoding='utf-8')
    print("Запись в файл df_soft_soft успешна!")
    soft_hard.to_csv('df_soft_hard', encoding='utf-8')
    print("Запись в файл df_soft_hard успешна!")
    profession_soft.to_csv('df_profession_soft',encoding='utf-8')
    print("Запись в файл df_profession_soft успешна!")
    nodes.to_csv('df_nodes',encoding='utf-8')
    print("Запись в файл df_nodes успешна!")

In [22]:
number_of_edges=100

In [26]:
graphistry.register(api=3, protocol="https", server="hub.graphistry.com",
                    personal_key_id="ВАШ_KEY_ID", personal_key_secret="ВАЩ_KEY_SECRET")

In [27]:
# Подготовка данных
nodes_set = set(df_soft_soft['source']).union(set(df_soft_soft['target']))
nodes_df = pd.DataFrame({"node": list(nodes_set)})
nodes_df["frequency"] = nodes_df["node"].map(soft_freq)
nodes_df["type"] = "soft"

# Фильтрация слабых связей
filtered_edges = df_soft_soft[df_soft_soft["npmi"] > 0.2] \
    .sort_values("npmi", ascending=False) \
    .head(number_of_edges)

top_nodes = set(filtered_edges['source']).union(set(filtered_edges['target']))
nodes_df = nodes_df[nodes_df['node'].isin(top_nodes)]

# Построение графа
g = graphistry.nodes(nodes_df, "node") \
    .edges(filtered_edges, "source", "target") \
    .bind(
        #point_color="type",
        point_size="frequency",
        edge_weight="npmi",
        edge_title="co_occurrence"
    )
print("ВИЗУАЛИЗАЦИЯ Soft-Soft графа по ТОП-"+str(number_of_edges)+ " связям по NPMI")

g.plot()

ВИЗУАЛИЗАЦИЯ Soft-Soft графа по ТОП-100 связям по NPMI


In [28]:
# Подготовка данных
nodes_set = set(df_soft_hard['source']).union(set(df_soft_hard['target']))
nodes_df = pd.DataFrame({"node": list(nodes_set)})

# Определение типа узла и частоты
def get_node_type(node):
    if node.startswith("SOFT_"):
        return "soft", soft_freq.get(node, 1)
    return "hard", hard_freq.get(node, 1)

nodes_df[["type", "frequency"]] = nodes_df["node"].apply(
    lambda x: pd.Series(get_node_type(x))
)

# Фильтрация связей
filtered_edges = df_soft_hard[df_soft_hard["npmi"] > 0.2] \
    .sort_values("npmi", ascending=False) \
    .head(number_of_edges)

top_nodes = set(filtered_edges['source']).union(set(filtered_edges['target']))
nodes_df = nodes_df[nodes_df['node'].isin(top_nodes)]

# Построение графа
g = graphistry.nodes(nodes_df, "node") \
    .edges(filtered_edges, "source", "target") \
    .bind(
        point_color="type",
        point_size="frequency",
        point_title="node",
        edge_weight="npmi",
        edge_title="co_occurrence"
    )\
    .encode_point_color("type", categorical_mapping={
          'soft': 'blue',
          'hard': 'orange'
      }, default_mapping='gray')


print("ВИЗУАЛИЗАЦИЯ Soft-Hard графа по ТОП-"+str(number_of_edges)+ " связям по NPMI")
g.plot()

ВИЗУАЛИЗАЦИЯ Soft-Hard графа по ТОП-100 связям по NPMI


In [29]:
# Подготовка данных
nodes_set = set(df_profession_soft['source']).union(set(df_profession_soft['target']))
nodes_df = pd.DataFrame({"node": list(nodes_set)})

# Определение типа узла и частоты
def get_node_type(node):
    if node in profession_freq:
        return "profession", profession_freq.get(node, 1)
    return "soft", soft_freq.get(node, 1)

nodes_df[["type", "frequency"]] = nodes_df["node"].apply(
    lambda x: pd.Series(get_node_type(x))
)

# Фильтрация связей
#filtered_edges = df_profession_soft[df_profession_soft["npmi"] > 0.05]
filtered_edges = df_profession_soft[df_profession_soft["npmi"] > 0.2] \
    .sort_values("npmi", ascending=False) \
    .head(number_of_edges)

top_nodes = set(filtered_edges['source']).union(set(filtered_edges['target']))
nodes_df = nodes_df[nodes_df['node'].isin(top_nodes)]
# Построение графа
g = graphistry.nodes(nodes_df, "node") \
    .edges(filtered_edges, "source", "target") \
    .bind(
        point_color="type",
        point_size="frequency",
        point_title="node",
        edge_weight="npmi",
        edge_title="co_occurrence"
    )\
    .encode_point_color("type", categorical_mapping={
          'profession': 'red',
          'soft': 'blue'
      }, default_mapping='gray')


print("ВИЗУАЛИЗАЦИЯ Profession-Soft графа по ТОП-"+str(number_of_edges)+ " связям по NPMI")
g.plot()

ВИЗУАЛИЗАЦИЯ Profession-Soft графа по ТОП-100 связям по NPMI
